# Memory( 대화 내용 기억)
- 대화 내용 기억
- LLM은 기본적으로 대화 내용을 기억하지 않는다.(stateless)
- 이전 대화내용을 게속 프롬프트에 주입해야 함
1. Short-Term Mermory
    - 단기기억: 한 대화 세션에 대한 기억
1. Long-Term Memory
    - 장기기억: 전체 세션에서 추출한 중요한 정보

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

prompt = ChatPromptTemplate(
    [
        ('system', '넌 유용한 챗봇이야'),
        MessagesPlaceholder(variable_name='chat_history'),  # 기존 채팅 내역을 다 주입
        ('human', '{input}'),
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

# 메모리를 저장할 변수는 {}다. 기존에 대화내용이 있다면 불러와라
memory.load_memory_variables({})

C:\Users\mingyu\AppData\Local\Temp\ipykernel_26880\3956690423.py:16: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')


{'chat_history': []}

In [4]:
from operator import itemgetter
runnable = RunnablePassthrough.assign(
    chat_history = RunnableLambda(memory.load_memory_variables)|
    itemgetter('chat_history')
)

In [5]:
chain = runnable | prompt | llm

In [6]:
res = chain.invoke({'input':'만나서 반가워'})

In [7]:
res.content

'저도 만나서 반가워요! 어떻게 도와드릴까요?'

In [8]:
my_message = '내이름이 뭐라고?'
res = chain.invoke({'input':my_message})
memory.save_context(
    {'human':my_message},
    {'ai': res.content}
)

In [9]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='내이름이 뭐라고?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='죄송하지만, 제가 당신의 이름을 알지 못해요. 이름을 알려주시면 기억해둘게요!', additional_kwargs={}, response_metadata={})]}

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import time
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

llm = ChatOpenAI(model = 'gpt-4.1-nano',temperature = 0)
prompt = ChatPromptTemplate(
    [
        ('system','넌 좀 틱틱대는 챗봇이야.'),
        MessagesPlaceholder(variable_name = 'chat_history'),
        ('human','{input}')
    ]
)
#Memory
memory = ConversationBufferMemory(return_messages = True, memory_key = 'chat_history')
runnable = RunnablePassthrough.assign(
    chat_history = RunnableLambda(memory.load_memory_variables)|
    itemgetter('chat_history')
)
chain = runnable | prompt | llm |StrOutputParser()
def stream_msg(chain, msg):
    full_msg = ''
    print('AI:',end='')
    for token in chain.stream({'input':msg}):
        full_msg += token
        print(token, end = '', flush=True)
    print('\n---')
    return full_msg



C:\Users\mingyu\AppData\Local\Temp\ipykernel_19500\3433334491.py:19: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages = True, memory_key = 'chat_history')


# Tavily Web Search
- LLM에게 웹검색을 쥐어주자

In [1]:
from dotenv import load_dotenv
from pprint import pprint
load_dotenv()

True

In [4]:
%pip install -q langchain_tavily

Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain_tavily import TavilySearch
search_tool = TavilySearch(
    max_results = 5,
    topic='general',
)

In [ ]:
res = search_tool.invoke({'query':'강남신세계에서 밥먹을만한 곳 알려줘'})
res['results']


[{'content': '강남신세계백화점 레스토랑맛집 (9곳) · 1. 더 마고 그릴 반포 · 2. 모던눌랑 강남고속터미널 · 3. 에토레 '
             '파미에스테이션 · 4. 쓰리버즈 센트럴시티점 · 5. 더 키친 일뽀르노',
  'raw_content': None,
  'score': 0.7062922,
  'title': "'강남신세계백화점 레스토랑' 맛집 빅데이터 추천순위 Top9 - 다이닝코드",
  'url': 'https://www.diningcode.com/list.dc?query=%EA%B0%95%EB%82%A8%EC%8B%A0%EC%84%B8%EA%B3%84%EB%B0%B1%ED%99%94%EC%A0%90%20%EB%A0%88%EC%8A%A4%ED%86%A0%EB%9E%91'},
 {'content': '신세계 팩토리스토어 신세계 스타일마켓 신세계 푸드마켓 * 신세계 브랜드 신세계 아카데미;) 신세계 온 아카데미 '
             '신세계 문화홀 신세계 VIP 클럽 + 혜택 안내 + 신세계 카드 * 신세계 아카데미 * 신세계 아카데미 * 신세계 '
             '문화홀 * 신세계 VIP 클럽 MY 신세계 MY 신세계 정상영업 10:30~20:00 연장영업 10:30~20:30 '
             '식당가영업 11:00~21:30 식당가 주문 마감 폐점 30분 전(단, 재료 소진 시 조기 마감 가능)  [하우스 '
             '오브 신세계] 주중/주말 10:30~22:00 \\* 주문 마감 1시간 전(단, 재료 소진 시 조기 마감 가능)  '
             '[B1 까사빠보/렌위치/브라우터/파이브가이즈] 10:30-21:00 (연장 영업 10:30-21:30), [B1 '
             '%아라비카] 10:30-21:00 [신세계 팩토리스토어 강남점] 주중/주말 11:00~21:00  [신세계 푸드마켓 '
             '도곡점] 주중/주말 10:30~22:00 (